In [17]:
import csv
from whistle_data_apis import get_pet_id, get_health_trends, get_wellness_index, get_sleeping_data, get_scratching_data, get_licking_data, get_drinking_data, get_eating_daily_trends, get_dailies, auth_token, get_daily_activity
from datetime import datetime, timedelta
from dateutils import get_day_number, get_date_from_day_number

# The number of days to go back to collect data for. days_back = 0 is yesterday
days_back = 3

# Print the data 
print_data = False

serial_numbers = [
    "AM6-7117F3F",
    "AM6-7117F2A",
    "AM6-7117F3B",  # MISSING
    "AM6-7117F29",
    "AM6-7117EC2",
    "AM6-7117F26",
    "AM6-7117F9C",  # BROKEN
    "AM6-7117F90",
    "AM6-7117EBD",
    "AM6-7117F8C",
    "AM6-7117F09",
    #"AMG-7117F76", # Not working
    "AM6-7117F77",
    "AM6-7117F8F",
    "AM6-7117F33",
    "AM6-7117F40",
    "AM6-7117F1E",
    "AM6-7117F72",
    "AM6-7117F98",
    "AM6-7117F5F",
    "AM6-7117F3D",
    "AM6-7117F48",
    "AM6-7117F2C",
    "AM6-7117F42",
    "AM6-7117F30",
    "AM6-7117F55",
    "AM6-7117E38",
    "AM6-7117F31",
    "AM6-7117F20",
    "AM6-7117F3E",
    "AM6-7117F2E",
    "AM6-7117F1F",
    "AM6-7117F8B",
    "AM6-7117F71",
    "AM6-7117F59",
    "AM6-7117F5B",
    "AM6-7117F2B",
    "AM6-7117F34",
    "AM6-7117F6E"
]

start_date = datetime.now() - timedelta(days=days_back)
start_date_formatted = start_date.strftime('%Y-%m-%d')
print("Start date: "  + start_date_formatted)

end_date = datetime.now() - timedelta(days=1)
end_date_formatted = end_date.strftime('%Y-%m-%d')
today_formatted = datetime.now().strftime('%Y-%m-%d')
print("End date: " + end_date_formatted)


## Create behaviors file
with open(f"whistledata-behaviors-{start_date_formatted}-{end_date_formatted}.csv", 'w', newline='') as behaviors_file, open(f"whistledata-durations-{start_date_formatted}-{end_date_formatted}.csv", 'w', newline='') as durations_file, open(f"whistledata-thresholds-{today_formatted}.csv", 'w', newline='') as thresholds_file, open(f"whistledata-hourly-activity-{start_date_formatted}-{end_date_formatted}.csv", 'w', newline='') as hourly_activity_file:
     
    ## Create the behaviors csv
    behaviors_writer = csv.writer(behaviors_file)
    behaviors_fields = [
        "id", "serial_number", "date", "wellness_score", "wellness_status", "sleeping_duration", "sleeping_start_datetime", "sleeping_end_datetime", 
        "scratching_score", "scratching_status", "licking_score", "licking_status", "drinking_score", "drinking_status", "eating_score",
        "eating_status", "minutes_active", "minutes_rest"
    ]
    behaviors_writer.writerow(behaviors_fields)

    ## Create the durations csv
    durations_writer = csv.writer(durations_file)
    durations_fields = [
        "id", "serial_number", "date", "start_datetime", "end_datetime", "duration" 
    ]
    durations_writer.writerow(durations_fields)

    ## Create the thresholds csv
    thresholds_writer = csv.writer(thresholds_file)
    thresholds_fields = [
        "id", "serial_number", "date", "category", "status", "min", "max", "percentile"
    ]
    thresholds_writer.writerow(thresholds_fields)

    ## Create the hourly activity csv
    hourly_activity_writer = csv.writer(hourly_activity_file)
    hourly_activity_fields = [
        "id", "serial_number", "date", "hour", "activity", "rest", "distance", "calories"
    ]
    hourly_activity_writer.writerow(hourly_activity_fields)

    for serial_number in serial_numbers:
        print("Fetching data for serial number: " + serial_number)
        pet_id = get_pet_id(auth_token, serial_number)

        # wellness
        ## NOTE: We pull one extra day of wellness data as there seems to be a bug where the first day of the API call returns no wellness score
        wellness = get_wellness_index(auth_token, pet_id, days_back + 1)
        wellness_data = wellness['data'][::-1]

        # sleeping
        sleeping = get_sleeping_data(auth_token, pet_id, days_back)
        sleeping_data = sleeping['data']

        # scratching
        scratching = get_scratching_data(auth_token, pet_id, days_back)
        scratching_data = scratching['data'][::-1]

        # licking
        licking = get_licking_data(auth_token, pet_id, days_back)
        licking_data = licking['data'][::-1]

        # drinking
        drinking = get_drinking_data(auth_token, pet_id, days_back)
        drinking_data = drinking['data']

        # eating
        eating = get_eating_daily_trends(auth_token, pet_id, days_back)
        eating_data = eating['data']

        # dailies
        dailies = get_dailies(auth_token, pet_id, start_date_formatted, today_formatted)
        dailies_data = dailies['dailies'][::-1]

        #hourly activity
        # daily_activity_hourly_list = []
        # for i in range(days_back):
        #     date = start_date + timedelta(days=i)
        #     day_number = get_day_number(date)
        #     daily_activity = get_daily_activity(auth_token, pet_id, day_number)
        #     daily_activity_hourly_list.append(daily_activity['daily'])

        if print_data:
            print("Wellness data: " + wellness_data)
            print("Sleeping data: " + sleeping_data)
            print("Scratching data: " + scratching_data)
            print("Licking data: " + licking_data)
            print("Drinking data: " + drinking_data)
            print("Eating data: " + eating_data)
            print("Dailies data: " + dailies_data)

        ## CREATE THE BEHAVIORS CSV
        for i in range(days_back):
            date = start_date + timedelta(days=i)
            date_formatted = date.strftime('%Y-%m-%d')

            behaviors_row = []
            behaviors_row.append(pet_id)
            behaviors_row.append(serial_number)
            behaviors_row.append(date_formatted)
            behaviors_row.append(wellness_data[i + 1]['score'])
            behaviors_row.append(wellness_data[i + 1]['status'])
            behaviors_row.append(sleeping_data[i]['duration'])
            behaviors_row.append(sleeping_data[i]['start_datetime'])
            behaviors_row.append(sleeping_data[i]['end_datetime'])
            behaviors_row.append(scratching_data[i]['score'])
            behaviors_row.append(scratching_data[i]['status'])
            behaviors_row.append(licking_data[i]['score'])
            behaviors_row.append(licking_data[i]['status'])
            behaviors_row.append(drinking_data[i]['score'])
            behaviors_row.append(drinking_data[i]['status'])
            behaviors_row.append(eating_data[i]['score'])
            behaviors_row.append(eating_data[i]['status'])
            behaviors_row.append(dailies_data[i]['minutes_active'])
            behaviors_row.append(dailies_data[i]['minutes_rest'])
            behaviors_writer.writerow(behaviors_row)

        ## CREATE THE THRESHOLDS CSV
        for i in range(3):
            thresholds_row = []
            thresholds_row.append(pet_id)
            thresholds_row.append(serial_number)
            thresholds_row.append(today_formatted)
            thresholds_row.append('wellness')
            thresholds_row.append(wellness['status_thresholds'][i]['status'])
            thresholds_row.append(wellness['status_thresholds'][i]['min_value'])
            thresholds_row.append(wellness['status_thresholds'][i]['max_value'])
            thresholds_row.append(wellness['status_thresholds'][i]['percentile'])
            thresholds_writer.writerow(thresholds_row)


        for i in range(3):
            thresholds_row = []
            thresholds_row.append(pet_id)
            thresholds_row.append(serial_number)
            thresholds_row.append(today_formatted)
            thresholds_row.append('sleeping')
            thresholds_row.append(sleeping['status_thresholds'][i]['status'])
            thresholds_row.append(sleeping['status_thresholds'][i]['min_value'])
            thresholds_row.append(sleeping['status_thresholds'][i]['max_value'])
            thresholds_row.append(sleeping['status_thresholds'][i]['percentile'])
            thresholds_writer.writerow(thresholds_row)
        
        for i in range(4):
            thresholds_row = []
            thresholds_row.append(pet_id)
            thresholds_row.append(serial_number)
            thresholds_row.append(today_formatted)
            thresholds_row.append('scratching')
            thresholds_row.append(scratching['status_thresholds'][i]['status'])
            thresholds_row.append(scratching['status_thresholds'][i]['min_value'])
            thresholds_row.append(scratching['status_thresholds'][i]['max_value'])
            thresholds_row.append(scratching['status_thresholds'][i]['percentile'])
            thresholds_writer.writerow(thresholds_row)

        for i in range(4):
            thresholds_row = []
            thresholds_row.append(pet_id)
            thresholds_row.append(serial_number)
            thresholds_row.append(today_formatted)
            thresholds_row.append('licking')
            thresholds_row.append(licking['status_thresholds'][i]['status'])
            thresholds_row.append(licking['status_thresholds'][i]['min_value'])
            thresholds_row.append(licking['status_thresholds'][i]['max_value'])
            thresholds_row.append(licking['status_thresholds'][i]['percentile'])
            thresholds_writer.writerow(thresholds_row)

        for i in range(3):
            thresholds_row = []
            thresholds_row.append(pet_id)
            thresholds_row.append(serial_number)
            thresholds_row.append(today_formatted)
            thresholds_row.append('drinking')
            thresholds_row.append(drinking['status_thresholds'][i]['status'])
            thresholds_row.append(drinking['status_thresholds'][i]['min_value'])
            thresholds_row.append(drinking['status_thresholds'][i]['max_value'])
            thresholds_row.append(drinking['status_thresholds'][i]['percentile'])
            thresholds_writer.writerow(thresholds_row)

        for i in range(3):
            thresholds_row = []
            thresholds_row.append(pet_id)
            thresholds_row.append(serial_number)
            thresholds_row.append(today_formatted)
            thresholds_row.append('eating')
            thresholds_row.append(eating['status_thresholds'][i]['status'])
            thresholds_row.append(eating['status_thresholds'][i]['min_value'])
            thresholds_row.append(eating['status_thresholds'][i]['max_value'])
            thresholds_row.append(eating['status_thresholds'][i]['percentile'])
            thresholds_writer.writerow(thresholds_row)
        
        ## CREATE DURATIONS CSV
        for sleeping_data_entry in sleeping_data:
            for disruption in sleeping_data_entry['disruptions']:
                disruptions_row = []
                disruptions_row.append(pet_id)
                disruptions_row.append(serial_number)
                disruptions_row.append(sleeping_data['start_date'])
                disruptions_row.append(disruption['start_datetime'])
                disruptions_row.append(disruption['end_datetime'])
                disruptions_row.append(disruption['duration'])
                disruptions_writer.writerow(disruptions_row)

        ## CREATE HOURLY ACTIVITY CSV
        ## NOTE: This is commented out as the hourly activity API is not working
        # for daily_activity_hourly in daily_activity_hourly_list:
        #     for activity in daily_hourly_activity['hourly_activity']:
        #         hourly_activity_row = []
        #         hourly_activity_row.append(pet_id)
        #         hourly_activity_row.append(serial_number)
        #         hourly_activity_row.append(get_date_from_day_number(daily_activity_hourly['day_number']))
        #         hourly_activity_row.append(activity['activity'])
        #         hourly_activity_row.append(activity['rest'])
        #         hourly_activity_row.append(activity['distance'])
        #         hourly_activity_row.append(activity['calories'])

        


Start date: 2024-08-03
End date: 2024-08-05
